# Current

## Create test example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import string, random, toolz, pandas, importlib, sys, os

In [3]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    192160.000000
mean        259.199839
std         256.922932
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        3174.000000
dtype: float64

## Run current version

In [5]:
!git stash
!git checkout main
#importlib.reload(dimod)

Saved working directory and index state WIP on binary-polynomial: bc4521f added symbolic arithmetic refactored relabel_variables, to_binary, to_spin filter out 0 coefficient terms in polynomial construction
Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [6]:
from dimod import BinaryPolynomial as BPold
import dimod

In [7]:
BPold.__init__??

Signature: BPold.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms
        self._terms = terms = {}
        for term, bias in poly:

            fsterm = asfrozenset(term)

            # when SPIN-valued, s^2 == 1, so we need to handle that case
            # in BINARY, x^2 == x
            if len(fsterm) < len(term) and vartype is Vartype.SPIN:
                new = set()
                term = tuple(term)  # make sure it has .count
                for v in fsterm:
                    if term.count(v) % 2:
                        new.add(v)
                fsterm = frozenset(new)

            if fsterm in terms:
                terms[fsterm] += bias
            else:
                terms[fsterm] = bias

        self.vartype = 

In [8]:
%%prun -s cumtime -l 20
bp = BPold(terms, 'SPIN')

         9711530 function calls (9711450 primitive calls) in 48.033 seconds

   Ordered by: cumulative time
   List reduced from 47 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   48.033   48.033 {built-in method builtins.exec}
        1    0.000    0.000   48.033   48.033 <string>:1(<module>)
        1    0.000    0.000   48.033   48.033 decorators.py:289(new_f)
        1    7.909    7.909   48.033   48.033 polynomial.py:89(__init__)
  5482062   38.220    0.000   38.220    0.000 {method 'count' of 'tuple' objects}
   192160    1.075    0.000    1.135    0.000 polynomial.py:32(asfrozenset)
  3460625    0.690    0.000    0.690    0.000 {method 'add' of 'set' objects}
   384322    0.079    0.000    0.079    0.000 {built-in method builtins.len}
   192170    0.060    0.000    0.060    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
    

In [9]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,181561.000000,181561.000000
mean,18.931703,1.058377
std,7.552280,8.721284
min,1.000000,1.000000
25%,13.000000,1.000000
50%,20.000000,1.000000
75%,25.000000,1.000000
max,42.000000,3677.000000


In [10]:
term = string.ascii_letters[:12]

In [11]:
%timeit BPold({term: 1}, 'SPIN').to_binary().to_spin() == BPold({term: 1}, 'SPIN')
BPold({term: 1}, 'SPIN').to_binary().to_spin() == BPold({term: 1}, 'SPIN')

1.14 s ± 310 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [12]:
%timeit BPold({term: 1}, 'BINARY').to_spin().to_binary() == BPold({term: 1}, 'BINARY')
BPold({term: 1}, 'BINARY').to_spin().to_binary() == BPold({term: 1}, 'BINARY')

1.22 s ± 265 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [13]:
len(BPold({term: 1}, 'SPIN').to_binary().to_spin())

4096

In [14]:
len(BPold({term: 1}, 'BINARY').to_spin().to_binary())

4096

In [15]:
prun -s cumtime -l 20 bp2 = bp.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         19196037 function calls (19196032 primitive calls) in 21.734 seconds

   Ordered by: cumulative time
   List reduced from 66 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   21.734   21.734 {built-in method builtins.exec}
        1    0.000    0.000   21.733   21.733 <string>:1(<module>)
      2/1   15.297    7.648   21.733   21.733 polynomial.py:232(relabel_variables)
  7237640    2.147    0.000    2.887    0.000 polynomial.py:256(<genexpr>)
   544686    1.648    0.000    2.116    0.000 _collections_abc.py:742(__iter__)
  6874522    0.740    0.000    0.740    0.000 {method 'get' of 'dict' objects}
   363120    0.462    0.000    0.608    0.000 polynomial.py:158(__setitem__)
        1    0.000    0.000    0.517    0.517 polynomial.py:177(copy)
        1    0.000    0.000    0.517    0.517 decorators.py:289(new_f)
        1    0.200    0.200    0.517    0.517 polynomial.py:89(__init__)
  1452484 

In [16]:
prun -s cumtime -l 20 bp3 = bp2.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         19196034 function calls (19196029 primitive calls) in 18.191 seconds

   Ordered by: cumulative time
   List reduced from 63 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   18.191   18.191 {built-in method builtins.exec}
        1    0.000    0.000   18.191   18.191 <string>:1(<module>)
      2/1    9.913    4.957   18.191   18.191 polynomial.py:232(relabel_variables)
  7237640    2.937    0.000    3.940    0.000 polynomial.py:256(<genexpr>)
   544686    1.989    0.000    2.508    0.000 _collections_abc.py:742(__iter__)
  6874522    1.003    0.000    1.003    0.000 {method 'get' of 'dict' objects}
   363120    0.674    0.000    0.886    0.000 polynomial.py:158(__setitem__)
  1452484    0.430    0.000    0.592    0.000 polynomial.py:32(asfrozenset)
   363120    0.404    0.000    0.565    0.000 polynomial.py:120(__delitem__)
   544683    0.350    0.000    0.519    0.000 polynomial.py:149(__getit

In [17]:
bp == bp2

False

In [18]:
bp == bp3

True

In [19]:
bp2 == bp3

False

# Exit interpreter
I just can't get module reloading to work properly here ...

In [20]:
print('BYE')
exit()
raise SystemExit
print('HI')

BYE


SystemExit: 

/home/daniel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Proposed

## Create test example

In [1]:
import string, random, toolz, pandas, importlib, sys, os, cytoolz, toolz, dimod

<function _binary_product at 0x7f35ef682790>


In [2]:
import itertools as it

In [3]:
%time terms = [(term, 1) for term in ''.join(random.choices(string.ascii_letters, k=10**6)).split('Z')]

CPU times: user 147 ms, sys: 606 µs, total: 148 ms
Wall time: 147 ms


In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    19393.000000
mean        50.565049
std         51.640829
min          0.000000
25%         14.000000
50%         35.000000
75%         69.000000
max        548.000000
dtype: float64

## Run proposed version

In [5]:
!mv polynomial-perf.ipynb{,.bak}
!git checkout binary-polynomial
!git stash pop
#!touch -c ./dimod/higherorder/polynomial.py
#importlib.reload(dimod)

dimod/higherorder/polynomial.py: needs merge
polynomial-perf.ipynb: needs merge
error: you need to resolve your current index first
No stash entries found.


In [6]:
from dimod import BinaryPolynomial as BPnew

In [7]:
BPnew.__init__??

Signature: BPnew.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms

        self._terms = terms = defaultdict(int)
        for term, bias in poly:
            terms[freeze_term(term, vartype)] += bias
        
        self.vartype = vartype
File:      ~/dimod/dimod/higherorder/polynomial.py
Type:      function


In [8]:
%%prun -s cumtime -l 20 
bp = BPnew(terms, 'SPIN')

         406093 function calls (406013 primitive calls) in 0.221 seconds

   Ordered by: cumulative time
   List reduced from 46 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.221    0.221 {built-in method builtins.exec}
        1    0.000    0.000    0.221    0.221 <string>:1(<module>)
        1    0.000    0.000    0.221    0.221 decorators.py:289(new_f)
        1    0.016    0.016    0.221    0.221 polynomial.py:108(__init__)
    19393    0.152    0.000    0.205    0.000 polynomial.py:40(freeze_term)
   347711    0.050    0.000    0.050    0.000 polynomial.py:47(<genexpr>)
    19403    0.002    0.000    0.002    0.000 {built-in method builtins.isinstance}
    19394    0.002    0.000    0.002    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck}


In [9]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,18602.000000,18602.000000
mean,17.624341,1.042522
std,7.647118,2.870007
min,0.000000,1.000000
25%,12.000000,1.000000
50%,19.000000,1.000000
75%,24.000000,1.000000
max,38.000000,389.000000


In [10]:
term = string.ascii_letters[:12]

In [11]:
%timeit BPnew({term: 1}, 'SPIN').to_binary().to_spin() == BPnew({term: 1}, 'SPIN')
BPnew({term: 1}, 'SPIN').to_binary().to_spin() == BPnew({term: 1}, 'SPIN')

887 ms ± 162 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [12]:
%timeit BPnew({term: 1}, 'BINARY').to_spin().to_binary() == BPnew({term: 1}, 'BINARY')
BPnew({term: 1}, 'BINARY').to_spin().to_binary() == BPnew({term: 1}, 'BINARY')

1.25 s ± 226 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [13]:
len(BPnew({term: 1}, 'BINARY').to_spin().to_binary())

4096

In [14]:
len(BPnew({term: 1}, 'SPIN').to_binary().to_spin())

4096

In [15]:
#len(BPnew({term: 1}, 'BINARY').to_spin() * BPnew({term: 1}, 'BINARY').to_spin())

In [16]:
prun -s cumtime -l 20 bp2 = bp.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         1832419 function calls (1832414 primitive calls) in 0.652 seconds

   Ordered by: cumulative time
   List reduced from 67 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.652    0.652 {built-in method builtins.exec}
        1    0.000    0.000    0.652    0.652 <string>:1(<module>)
      2/1    0.301    0.151    0.652    0.652 polynomial.py:236(relabel_variables)
   692900    0.152    0.000    0.202    0.000 polynomial.py:260(<genexpr>)
    55809    0.033    0.000    0.063    0.000 _collections_abc.py:742(__iter__)
   655700    0.051    0.000    0.051    0.000 {method 'get' of 'dict' objects}
    37202    0.025    0.000    0.035    0.000 polynomial.py:162(__setitem__)
   130210    0.024    0.000    0.031    0.000 polynomial.py:35(asfrozenset)
        1    0.000    0.000    0.030    0.030 polynomial.py:181(copy)
        1    0.000    0.000    0.030    0.030 decorators.py:289(new_f)
    55806 

In [17]:
prun -s cumtime -l 20 bp3 = bp2.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         1832416 function calls (1832411 primitive calls) in 0.594 seconds

   Ordered by: cumulative time
   List reduced from 64 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.594    0.594 {built-in method builtins.exec}
        1    0.000    0.000    0.594    0.594 <string>:1(<module>)
      2/1    0.227    0.113    0.594    0.594 polynomial.py:236(relabel_variables)
   692900    0.154    0.000    0.208    0.000 polynomial.py:260(<genexpr>)
    55809    0.036    0.000    0.069    0.000 _collections_abc.py:742(__iter__)
   655700    0.054    0.000    0.054    0.000 {method 'get' of 'dict' objects}
    37202    0.027    0.000    0.037    0.000 polynomial.py:162(__setitem__)
        1    0.000    0.000    0.033    0.033 polynomial.py:181(copy)
        1    0.000    0.000    0.033    0.033 decorators.py:289(new_f)
   130210    0.024    0.000    0.033    0.000 polynomial.py:35(asfrozenset)
        1 

In [18]:
bp == bp2

False

In [19]:
bp == bp3

True

In [20]:
bp2 == bp3

False

In [21]:
import numpy as np
%load_ext line_profiler

In [22]:
#%lprun -fdimod.make_quadratic  bqm = dimod.make_quadratic(bp, vartype = 'BINARY', strength = 2**6)

In [23]:
import functools as ft
import itertools as it
import cytoolz as tl
from dimod.higherorder.utils import _new_product, as_vartype
from collections import defaultdict

In [24]:
%%time
idx = defaultdict(
    dict,
    tl.valmap(
        tl.compose(dict, ft.partial(map, tl.second)),
        tl.groupby(
            tl.first,
            (
                (frozenset(pair), item)
                for item in bp.items()
                if len(term := tl.first(item)) >= 3
                for pair in it.combinations(term, 2) ))))

CPU times: user 18.1 s, sys: 1.19 s, total: 19.3 s
Wall time: 19.3 s


In [25]:
len(idx)

1275

In [26]:
#idx

In [27]:
#bp

In [28]:
#terms

In [29]:
que = tl.valmap(tl.compose(set, ft.partial(map, tl.first)), tl.groupby(tl.second, tl.valmap(len, idx).items()))
most = max(que)
most

2720

In [30]:
que[most].pop()

frozenset({'F', 'J'})

In [31]:
que[most]

set()

In [32]:
constraints = []
reduced_terms = [item for item in bp.items() if len(tl.first(item)) <= 2]
pair, terms = tl.topk(1, idx.items(), tl.compose(len,tl.second))[0]
p = _new_product(bp.variables, *pair)
constraints.append((pair, p))
del idx[pair]
pset = {p}

In [33]:
for term, bias in terms.items():
    for pair2 in map(frozenset, (it.combinations(term, 2))):
        if pair2 != pair:
            del idx[pair2][term]
    base_term = term - pair
    new_term = base_term | pset
    if len(new_term) <= 3:
        reduced_terms.append((new_term, bias))
    else:
        for pair3 in map(frozenset, it.combinations(new_term, 2)):
            idx[pair3][new_term] = bias

In [34]:
#reduced_terms

In [35]:
constraints

[(frozenset({'F', 'J'}), 'J*F')]

In [36]:
#terms

In [37]:
tl.topk(5, tl.valmap(len, idx).items(), tl.second)

((frozenset({'K', 'Y'}), 2689),
 (frozenset({'A', 'T'}), 2680),
 (frozenset({'C', 'l'}), 2674),
 (frozenset({'C', 'p'}), 2671),
 (frozenset({'L', 'M'}), 2670))

In [38]:
def reduce_terms(bp):
    variables = bp.variables
    constraints = []
    reduced_terms = [item for item in bp.items() if len(tl.first(item)) <= 2]

    idx = defaultdict(
        dict,
        tl.valmap(
            tl.compose(dict, ft.partial(map, tl.second)),
            tl.groupby(
                tl.first,
                (
                    (frozenset(pair), item)
                    for item in bp.items()
                    if len(term := tl.first(item)) >= 3
                    for pair in it.combinations(term, 2) ))))

    que = defaultdict(
        set,
        tl.valmap(
            tl.compose(set, ft.partial(map, tl.first)),
            tl.groupby(tl.second, tl.valmap(len, idx).items())))

    while idx:
        new_pairs = set()
        most = max(que)
        
        #print(most, que[most])
        #_, (pair, terms) = max((len(tl.second(item)), item) for item in idx.items())
        #pair, terms = max(idx.items(), key=tl.compose(len,tl.second))
        
        que_most = que[most]
        pair = que_most.pop()
        if not que_most:
            del que[most]
        terms = idx[pair]
        
        p = _new_product(variables, *pair)
        constraints.append((pair, p))
        del idx[pair]
        pset = {p}

        for term, bias in terms.items():
            for pair2 in map(frozenset, (it.combinations(term, 2))):
                if pair2 != pair:
                    if pair2 & pair:
                        count = len(idx[pair2])
                        que_count = que[count]
                        que_count.remove(pair2)
                        if not que_count:
                            del que[count]
                        if count > 1:
                            que[count - 1].add(pair2)
                    idx_pair2 = idx[pair2]
                    del idx_pair2[term]
                    if not idx_pair2:
                        del idx[pair2]
    
            new_term = (term - pair) | pset
            if len(new_term) <= 2:
                reduced_terms.append((new_term, bias))
            else:
                for new_pair in map(frozenset, it.combinations(new_term, 2)):
                    idx[new_pair][new_term] = bias
                    if p in new_pair:
                        new_pairs.add(new_pair)
                        
        for new_pair in new_pairs:
            que[len(idx[new_pair])].add(new_pair)

    return reduced_terms, constraints

In [39]:
%prun -s cumtime -l 30 reduced_terms, constraints = reduce_terms(bp)

         25411126 function calls in 70.785 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   70.785   70.785 {built-in method builtins.exec}
        1    0.004    0.004   70.785   70.785 <string>:1(<module>)
        1   56.359   56.359   70.781   70.781 4179918042.py:1(reduce_terms)
  3268667   10.024    0.000   10.063    0.000 4179918042.py:12(<genexpr>)
  5919520    2.057    0.000    2.057    0.000 {method 'remove' of 'set' objects}
  8977662    1.580    0.000    1.580    0.000 {method 'add' of 'set' objects}
  6724636    0.548    0.000    0.548    0.000 {built-in method builtins.len}
    58933    0.045    0.000    0.079    0.000 utils.py:88(_new_product)
    37206    0.021    0.000    0.053    0.000 _collections_abc.py:742(__iter__)
    58933    0.036    0.000    0.036    0.000 {built-in method builtins.max}
    37204    0.020    0.000    0.033    0.000 polynomial.py:153(__getitem__)
    589

In [40]:
len(reduced_terms), len(constraints)

(18602, 58933)

In [41]:
#%prun -s cumtime -l 30 bqm = dimod.make_quadratic(bp, 1000, 'BINARY')

In [42]:
import itertools
import warnings

from collections import Counter

import numpy as np


from dimod.higherorder.utils import _new_product, as_vartype, _binary_product, _new_aux, _spin_product
print(_binary_product)
from dimod.binary_quadratic_model import BinaryQuadraticModel
from dimod.higherorder.polynomial import BinaryPolynomial
from dimod.sampleset import as_samples
from dimod.vartypes import as_vartype, Vartype

def make_quadratic(poly, strength, vartype=None, bqm=None):
    """Create a binary quadratic model from a higher order polynomial.

    Args:
        poly (dict):
            Polynomial as a dict of form {term: bias, ...}, where `term` is a tuple of
            variables and `bias` the associated bias.

        strength (float):
            The energy penalty for violating the prodcut constraint.
            Insufficient strength can result in the binary quadratic model not
            having the same minimizations as the polynomial.

        vartype (:class:`.Vartype`/str/set, optional):
            Variable type for the binary quadratic model. Accepted input values:

            * :class:`.Vartype.SPIN`, ``'SPIN'``, ``{-1, 1}``
            * :class:`.Vartype.BINARY`, ``'BINARY'``, ``{0, 1}``

            If `bqm` is provided, `vartype` is not required.

        bqm (:class:`.BinaryQuadraticModel`, optional):
            The terms of the reduced polynomial are added to this binary quadratic model.
            If not provided, a new binary quadratic model is created.

    Returns:
        :class:`.BinaryQuadraticModel`

    Examples:

        >>> poly = {(0,): -1, (1,): 1, (2,): 1.5, (0, 1): -1, (0, 1, 2): -2}
        >>> bqm = dimod.make_quadratic(poly, 5.0, dimod.SPIN)

    """
    if vartype is None:
        if bqm is None:
            raise ValueError("one of vartype or bqm must be provided")
        else:
            vartype = bqm.vartype
    else:
        vartype = as_vartype(vartype)  # handle other vartype inputs
        if bqm is None:
            bqm = BinaryQuadraticModel.empty(vartype)
        else:
            bqm = bqm.change_vartype(vartype, inplace=False)

    # for backwards compatibility, add an info field
    if not hasattr(bqm, 'info'):
        bqm.info = {}
    bqm.info['reduction'] = {}

    if poly.vartype !=vartype:
        poly = BinaryPolynomial(poly, vartype=vartype)

    variables = set().union(*poly)
    reduced_terms, constraints = dimod.higherorder.utils.reduce_terms(poly)
  
    for (u, v), p in constraints:

        # add a constraint enforcing the relationship between p == u*v
        if vartype is Vartype.BINARY:
            constraint = _binary_product([u, v, p])
            bqm.info['reduction'][(u, v)] = {'product': p}
        elif vartype is Vartype.SPIN:
            aux = _new_aux(variables, u, v)  # need an aux in SPIN-space
            constraint = _spin_product([u, v, p, aux])
            bqm.info['reduction'][(u, v)] = {'product': p, 'auxiliary': aux}
        else:
            raise RuntimeError("unknown vartype: {!r}".format(vartype))

        # scale constraint and update the polynomial with it
        constraint.scale(strength)
        for v, bias in constraint.linear.items():
            bqm.add_variable(v, bias)
        for uv, bias in constraint.quadratic.items():
            bqm.add_interaction(*uv, bias)
        bqm.offset += constraint.offset

    for term, bias in reduced_terms:
        if len(term) == 2:
            bqm.add_interaction(*term , bias)
        elif len(term) == 1:
            bqm.add_variable(*term, bias)
        elif len(term) == 0:
            bqm.offset += bias
        else:
            # still has higher order terms, this shouldn't happen
            msg = ('Internal error: not all higher-order terms were reduced. '
                   'Please file a bug report.')
            raise RuntimeError(msg)

    return bqm

<function _binary_product at 0x7f35ef682790>


In [43]:
zz = set(range(4))

In [44]:
%lprun -fdimod.higherorder.utils.reduce_terms bqm = dimod.make_quadratic(BPnew({term: 1}, 'SPIN'), strength=5, vartype='SPIN')

Timer unit: 1e-06 s

Total time: 0.07269 s
File: /home/daniel/dimod/dimod/higherorder/utils.py
Function: reduce_terms at line 106

Line #      Hits         Time  Per Hit   % Time  Line Contents
   106                                           def reduce_terms(bp):
   107         1          6.0      6.0      0.0      variables = bp.variables
   108         1          1.0      1.0      0.0      constraints = []
   109         1         20.0     20.0      0.0      reduced_terms = [item for item in bp.items() if len(tl.first(item)) <= 2]
   110                                           
   111         2         24.0     12.0      0.0      idx = defaultdict(
   112         1          1.0      1.0      0.0          dict,
   113         2        644.0    322.0      0.9          tl.valmap(
   114         1         11.0     11.0      0.0              tl.compose(dict, ft.partial(map, tl.second)),
   115         2       1881.0    940.5      2.6              tl.groupby(
   116         1          0

In [45]:
%lprun -fdimod.higherorder.utils.reduce_terms bqm = dimod.make_quadratic(bp, strength=5, vartype=bp.vartype)

Timer unit: 1e-06 s

Total time: 364.994 s
File: /home/daniel/dimod/dimod/higherorder/utils.py
Function: reduce_terms at line 106

Line #      Hits         Time  Per Hit   % Time  Line Contents
   106                                               constraints = []
   107         1      12957.0  12957.0      0.0      reduced_terms = [item for item in bp.items() if len(tl.first(item)) <= 2]
   108         1          3.0      3.0      0.0  
   109         1      46129.0  46129.0      0.0      idx = defaultdict(
   110                                                   dict,
   111         2        458.0    229.0      0.0          tl.valmap(
   112         1          2.0      2.0      0.0              tl.compose(dict, partial(map, tl.second)),
   113         2    5879020.0 2939510.0      1.6              tl.groupby(
   114         1         26.0     26.0      0.0                  tl.first,
   115         2   17846109.0 8923054.5      4.9                  (
   116         1          2.0      

In [46]:
%time len(dimod.make_quadratic(BPnew({term: 1}, 'SPIN'), strength=5, vartype='SPIN'))

CPU times: user 13.2 ms, sys: 9 µs, total: 13.2 ms
Wall time: 12.1 ms


92

In [47]:
make_quadratic??

Signature: make_quadratic(poly, strength, vartype=None, bqm=None)
Source:   
def make_quadratic(poly, strength, vartype=None, bqm=None):
    """Create a binary quadratic model from a higher order polynomial.

    Args:
        poly (dict):
            Polynomial as a dict of form {term: bias, ...}, where `term` is a tuple of
            variables and `bias` the associated bias.

        strength (float):
            The energy penalty for violating the prodcut constraint.
            Insufficient strength can result in the binary quadratic model not
            having the same minimizations as the polynomial.

        vartype (:class:`.Vartype`/str/set, optional):
            Variable type for the binary quadratic model. Accepted input values:

            * :class:`.Vartype.SPIN`, ``'SPIN'``, ``{-1, 1}``
            * :class:`.Vartype.BINARY`, ``'BINARY'``, ``{0, 1}``

            If `bqm` is provided, `vartype` is not required.

        bqm (:class:`.BinaryQuadraticModel`, optional):

In [48]:
BinaryPolynomial.

SyntaxError: invalid syntax (3330683553.py, line 1)

In [ ]:
dimod.BQM

In [ ]:
itertools

In [ ]:
%prun -s cumtime -l 20 make_quadratic(bp, 100, vartype='BINARY')

In [ ]:
from dimod.higherorder import utils

In [ ]:
%pwd

In [ ]:
bool(set())

In [ ]:
bool